In [1]:
# This file is to plot our final results

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 5)) 
sns.set(style='whitegrid')

df = pd.read_csv('DATA.csv')
df.head()

,Unnamed: 0,Gold_GroundTruth,BITCOIN_GroundTruth,Gold_Prediction,BITCOIN_Prediction,test1g,test1b,Unnamed: 7
0,0,1308.35,609.11000,1125.7000,610.3800,0.435,0.8027,NaN
1,1,1314.85,610.19000,1308.3500,609.2223,0.435,0.8027,NaN
2,2,1313.80,608.66003,1311.2546,609.6876,0.435,0.8027,NaN
3,3,1326.10,598.88000,1312.3975,609.4840,0.435,0.8027,NaN
4,4,1339.10,597.42000,1315.7969,607.9764,0.435,0.8027,NaN


<Figure size 1440x360 with 0 Axes>

In [3]:
def value_computation(name, alpha_b=0.02, alpha_g=0.01 ,data=df):
    # enter your col name, alpha_b = 0.02, alpha_g = 0.01 for example
    # n_b n_g is the number of bitcoin and gol
    n_b = np.array(df[name+'b'])
    n_g = np.array(df[name+'g'])
    
    GP = np.array(df['Gold_Prediction'])
    GG = np.array(df['Gold_GroundTruth'])
    BP = np.array(df['BITCOIN_Prediction'])
    BG = np.array(df['BITCOIN_GroundTruth'])
    n = GG.shape[0]
    
    # Weight for Prediction
    value_P = np.zeros((n, 1))
    value_P[0] = 1000
    cash_P = np.zeros((n,1))
    cash_P[0] = 1000
    for day in range(1,n):
        del_B = n_b[day] - n_b[day-1]
        del_G = n_g[day] - n_g[day-1]
        if del_B > 0:
            change_B = - del_B*BP[day]*(1+alpha_b)
        else:
            change_B = - del_B*BP[day]*(1-alpha_b)
        if del_G > 0:
            change_G = - del_B*BP[day]*(1+alpha_g)
        else:
            change_G = - del_B*BP[day]*(1-alpha_g)
        cash_P[day] = cash_P[day-1] + change_B + change_G
        value_P[day] = cash_P[day] + n_b[day]*BP[day] + n_g[day]*GP[day]

    # Weight for Ground Truth
    value_G = np.zeros((n, 1))
    value_G[0] = 1000
    cash_G = np.zeros((n,1))
    cash_G[0] = 1000
    for day in range(1,n):
        del_B = n_b[day] - n_b[day-1]
        del_G = n_g[day] - n_g[day-1]
        if del_B > 0:
            change_B = - del_B*BG[day]*(1+alpha_b)
        else:
            change_B = - del_B*BG[day]*(1-alpha_b)
        if del_G > 0:
            change_G = - del_B*BG[day]*(1+alpha_g)
        else:
            change_G = - del_B*BG[day]*(1-alpha_g)
        cash_P[day] = cash_P[day-1] + change_B + change_G
        value_P[day] = cash_P[day] + n_b[day]*BG[day] + n_g[day]*GG[day]
    
    return value_P, value_G

In [8]:
def weight_plot(name, P, data=df):
    n_b = np.array(df[name+'b'])
    n_g = np.array(df[name+'g'])
    GG = np.array(df['Gold_GroundTruth'])
    BG = np.array(df['BITCOIN_GroundTruth'])
    
    w_b = BG * n_b / P
    w_g = GG * n_g / P
    w_c = 1 - w_b - w_g
    
    sns.lineplot(data=[w_c.flatten(),w_b.flatten(),w_g.flatten()])
    plt.xlabel('Day')
    plt.ylabel('Weight')
    plt.show()
    plt.savefig('../img/'+name+'W.png')

def value_plot(name, P):
    sns.lineplot(data=P.flatten(),color='black')
    plt.xlabel('Day')
    plt.ylabel('Value [$]')
    plt.show()
    plt.savefig('../img/'+name+'P.png')
    
def ratio_plot(name, P, G):
    rP = P[1:]/P[:-1].flatten()
    rG = G[1:]/P[:-1].flatten()
    sns.lineplot(data=[rP,rG])
    plt.xlabel('Day')
    plt.ylabel('Ratio')
    plt.show()
    plt.savefig('../img/'+name+'R.png')

In [10]:
name = 'test1'

P, G = value_computation(name)
weight_plot(name, P)
value_plot(name, P)
ratio_plot(name, P, G)